In [112]:
import spacy
import pytextrank
import json
from rake_nltk import Rake
import os

In [113]:
# Get all terms from index_terms.json
#  note. this gets all leaves in the json i.e. lowest level terms
term_tree = json.load(open("index_terms.json"))

def get_leafs(term_tree):
    terms = set()
    queue = [term_tree]

    while(queue):
        curr_dict = queue.pop()
        for key in curr_dict:
            if(curr_dict[key]):
                queue.append(curr_dict[key])
            else:
                terms.add(key.lower())
    return terms

terms = get_leafs(term_tree)

In [114]:
def topicRank(text):

# load a spaCy model, depending on language, scale, etc.
    nlp = spacy.load("en_core_web_sm")

    # add PyTextRank to the spaCy pipeline
    nlp.add_pipe("topicrank")
    doc = nlp(text)
    
    tr = doc._.textrank
        
    topics = set()

    for node in tr.node_list:
        for topic in node:
            topics.add(str(topic).lower())
            
    return topics

def rake(text):
    # print(text)
    r = Rake(min_length=1, max_length=8, include_repeated_phrases=False)
    r.extract_keywords_from_text(text)
    ranked_keywords = r.get_ranked_phrases()
    top_keywords = ranked_keywords[:5000]
    
    keywords = set()
    for keyword in top_keywords:
        keywords.add(str(keyword).lower())
    return keywords

In [115]:
# example text
# text = "Compatibility of systems of linear constraints over the set of natural numbers. Criteria of compatibility of a system of linear Diophantine equations, strict inequations, and nonstrict inequations are considered. Upper bounds for components of a minimal set of solutions and algorithms of construction of minimal generating sets of solutions for all types of systems are given. These criteria and the corresponding algorithms for constructing a minimal supporting set of solutions can be used in solving all the considered types systems and systems of mixed types."
doi = "10.1145.3351095.3372826"
text = open(f"dataset/{doi}/{doi}.txt").read()

In [116]:
topics = topicRank(text)

print(topics.intersection(terms))

KeyboardInterrupt: 

In [122]:
dataset_dir = "./dataset"
for doi in os.listdir(dataset_dir):
    text_path = os.path.join(dataset_dir, doi, f"{doi}.txt")
    if(not os.path.exists(text_path)):
        continue
    print()
    text = open(text_path).read()
    
    terms_path = os.path.join(dataset_dir, doi, f"{doi}.json")
    if(not os.path.exists(terms_path)):
        continue
    
    gt_term_tree = json.load(open(terms_path))
    gt_terms = get_leafs(gt_term_tree)
    
    topicRank_topics = topicRank(text)
    topicRank_terms = topicRank_topics.intserection(terms)
    
    
    rake_topics = rake(text)
    rake_terms = rake_topics.intersection(terms)
    
    print(rake_terms)
    print(gt_terms)

{'pointing', 'metrics', 'sociology', 'trees', 'law', 'social networks', 'abstraction', 'copyrights', 'design', 'frameworks', 'taxation', 'bioinformatics', 'women', 'men', 'measurement', 'patterns', 'medical records'}



UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 1024: character maps to <undefined>

In [121]:
json.load("dataset/10.1145.3351095.3372826/10.1145.3351095.3372826.json")

AttributeError: 'str' object has no attribute 'read'